In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_excel("zbior_punktow.xlsx", 'Arkusz2')
dct = dict()
pkt_nazwa = list(df['U'])
pkt_x = list(df['x'])
pkt_y = list(df['y'])
for i in range(len(pkt_x)):
    dct[pkt_nazwa[i]] = (pkt_x[i], pkt_y[i])

df2 = pd.read_excel("zbior_punktow.xlsx", 'Arkusz3')

A0_idx = df2.A0.to_list()
A1_idx = df2.A1.to_list()
A2_idx = df2.A2.to_list()
A3_idx = df2.A3.to_list()

A0 = []
A1 = []
A2 = []
A3 = []
for i in range(len(A0_idx)):
    A0.append(dct[A0_idx[i]])
    A1.append(dct[A1_idx[i]])
    A2.append(dct[A2_idx[i]])
    A3.append(dct[A3_idx[i]])


B0 = []


A1xmin = min([point[0] for point in A1])
A2xmax = max([point[0] for point in A2])

for point_name in dct.keys():
    if A1xmin < dct[point_name][0] < A2xmax:
        if (dct[point_name] not in A1) and (dct[point_name] not in A2) and (dct[point_name] not in A0) and (dct[point_name] not in A3):
            B0.append(dct[point_name])

In [18]:
def oblicz_d(A1_point,A2_point):
    x_A1, y_A1 = A1_point
    x_A2, y_A2 = A2_point
    d1 = (x_A2 - x_A1)/2
    d2 = (y_A2 - y_A1)/2
    print(d1,d2)
    if d1 > d2:
        d1, d2 = d2, d1
    return min(d1,d2)

def stworz_krzywa(A1_point,A2_point):
    d = oblicz_d(A1_point,A2_point)
    x_A1, y_A1 = A1_point
    x_A2, y_A2 = A2_point
    f1 = (x_A1+d,y_A1+d) 
    f2 = (x_A2-d,y_A2-d)
    return (A1_point,f1,f2,A2_point)

def oblicz_odleglosc(u,A1_point,A2_point):
    A1_point, f1, f2, A2_point = stworz_krzywa(A1_point,A2_point)
    print(A1_point, f1, f2, A2_point)
    lista = [A1_point, f1, f2, A2_point]
    x_u, y_u = u
    odle = []
    for i in range(3):
        if lista[i][0]-lista[i+1][0] == 0:
            A = 0
            B = -1
            C = 0
        else: 
            B = -1
            A = (lista[i][1]-lista[i+1][1])/(lista[i][0]-lista[i+1][0])
            C = lista[i][1] - lista[i][0]*(lista[i][1]-lista[i+1][1])/(lista[i][0]-lista[i+1][0])
        odle.append((abs(A*x_u + B*y_u + C))/(A**2+B**2)**(1/2))
    return min(odle),odle.index(min(odle)), f1, f2

# print(oblicz_d(dct["N22"],dct['N12']))
# print(oblicz_odleglosc(dct['N22'],dct['N12'],dct['N5']))

def odleglosc(point_1,point_2):
    x_1, y_1 = point_1
    x_2, y_2 = point_2
    return ((x_2-x_1)**2 + (y_2 - y_1)**2)**(1/2)

def oblicz_wspolczynnik_skoringowy(u,A1_point,A2_point):
    odle, idx, f1, f2 = oblicz_odleglosc(u,A1_point,A2_point)
    odleglosci = [odleglosc(A1_point,f1),odleglosc(f1,f2),odleglosc(f2,A2_point)]
    suma = odleglosci[0] + odleglosci[1] + odleglosci[2]
    waga = odleglosci[idx]/suma
    wsp_skoringowy = odle * waga
    return wsp_skoringowy

ranking = []

for point in B0:
    suma = 0
    for A1_point in A1:
        for A2_point in A2:
            suma += oblicz_wspolczynnik_skoringowy(point,A1_point,A2_point)
    point = list(dct.keys())[list(dct.values()).index((point[0], point[1]))]
    ranking.append((point,suma))

ranking = sorted(ranking, key = lambda tup: tup[1], reverse=False)

print(ranking)

13.0 10.5
(6, 14) (16.5, 24.5) (21.5, 24.5) (32, 35)
14.5 9.5
(6, 14) (15.5, 23.5) (25.5, 23.5) (35, 33)
15.0 6.5
(6, 14) (12.5, 20.5) (29.5, 20.5) (36, 27)
16.0 4.0
(6, 14) (10.0, 18.0) (34.0, 18.0) (38, 22)
12.5 12.0
(7, 11) (19.0, 23.0) (20.0, 23.0) (32, 35)
14.0 11.0
(7, 11) (18.0, 22.0) (24.0, 22.0) (35, 33)
14.5 8.0
(7, 11) (15.0, 19.0) (28.0, 19.0) (36, 27)
15.5 5.5
(7, 11) (12.5, 16.5) (32.5, 16.5) (38, 22)
11.0 13.0
(10, 9) (21.0, 20.0) (21.0, 24.0) (32, 35)
12.5 12.0
(10, 9) (22.0, 21.0) (23.0, 21.0) (35, 33)
13.0 9.0
(10, 9) (19.0, 18.0) (27.0, 18.0) (36, 27)
14.0 6.5
(10, 9) (16.5, 15.5) (31.5, 15.5) (38, 22)
8.5 15.0
(15, 5) (23.5, 13.5) (23.5, 26.5) (32, 35)
10.0 14.0
(15, 5) (25.0, 15.0) (25.0, 23.0) (35, 33)
10.5 11.0
(15, 5) (25.5, 15.5) (25.5, 16.5) (36, 27)
11.5 8.5
(15, 5) (23.5, 13.5) (29.5, 13.5) (38, 22)
13.0 10.5
(6, 14) (16.5, 24.5) (21.5, 24.5) (32, 35)
14.5 9.5
(6, 14) (15.5, 23.5) (25.5, 23.5) (35, 33)
15.0 6.5
(6, 14) (12.5, 20.5) (29.5, 20.5) (36, 27)
16.0